Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
!unzip data.zip
!pip install transformers
!pip install datasets

--2021-10-25 08:43:06--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-25 08:43:06--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   116KB/s    in 1.8s    

2021-10-25 08:43:09 (116 KB/s) - ‘data.zip’ saved [217313/217313]

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creat

In [12]:
import os
import time
import random
import datetime
import numpy as np
import pandas
from pdb import set_trace
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [13]:
## Various utility functions
class args_class:
    dataset = "data/"
    model_name = "bert-base-uncased"
    gpu = 0

def set_seed(seed=4):
	# fixing seeds to reproduce results (exact!)
	torch.backends.cudnn.deterministic = True
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)

def parse_args():
	return args_class()

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

In [14]:
def process_data(path, train=True, test=False):
    if test == True:
        x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
        word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
        word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
        x_test = x_test.drop("position", axis=1)
        x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
        x_test.insert(3, "word1", word1)
        x_test.insert(5, "word2", word2)
        # x_test.to_csv("./test.csv", header=None, index=False)
        return x_test
    
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

class myDataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, test=False, model_name='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train
        self.test = test

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train, test=test)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) # , do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
            max_length=self.max_len, padding='max_length', truncation=True)
        if self.test == False:
            y = torch.tensor(sample['label'])

        # format as required by Train
        item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
        if self.test == False:
            item['labels'] = y
        return item
    
    def __len__(self):
        return len(self.dataset_df)

In [15]:
set_seed()

metric = load_metric("accuracy")
def compute_metrics_fn(eval_prediction):
    logits, labels = eval_prediction
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

args = parse_args()
trainset = myDataset(args.dataset, max_len=100, train=True, model_name=args.model_name)
devset = myDataset(args.dataset, max_len=100, train=False, model_name=args.model_name)
model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=2)
print("resource preparation done: {}".format(datetime.datetime.now()))

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/

resource preparation done: 2021-10-25 09:31:11.851447


In [16]:
training_args = TrainingArguments(
    output_dir="2018EE10957_B_model", 
    overwrite_output_dir=True, 
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch", 
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=5,
    seed=4,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    warmup_steps=500,
    logging_dir='./logs',
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=trainset, eval_dataset=devset,
    compute_metrics=compute_metrics_fn
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 5428
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1700


Epoch,Training Loss,Validation Loss,Accuracy
1,0.626100,0.667261,0.656740
2,0.434600,0.644094,0.697492
3,0.233400,1.012652,0.691223
4,0.098700,1.481145,0.694357
5,0.038700,1.602394,0.710031


***** Running Evaluation *****
  Num examples = 638
  Batch size = 16
Saving model checkpoint to 2018EE10957_B_model/checkpoint-340
Configuration saved in 2018EE10957_B_model/checkpoint-340/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-340/pytorch_model.bin
Deleting older checkpoint [2018EE10957_B_model/checkpoint-680] due to args.save_total_limit
Deleting older checkpoint [2018EE10957_B_model/checkpoint-850] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 16
Saving model checkpoint to 2018EE10957_B_model/checkpoint-680
Configuration saved in 2018EE10957_B_model/checkpoint-680/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-680/pytorch_model.bin
Deleting older checkpoint [2018EE10957_B_model/checkpoint-340] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 16
Saving model checkpoint to 2018EE10957_B_model/checkpoint-1020
Configuration saved in 2018EE10957_

TrainOutput(global_step=1700, training_loss=0.2863165973214542, metrics={'train_runtime': 1018.6079, 'train_samples_per_second': 26.644, 'train_steps_per_second': 1.669, 'total_flos': 1394694148920000.0, 'train_loss': 0.2863165973214542, 'epoch': 5.0})

In [17]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10957_B_model.zip 2018EE10957_B_model

## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: 2018EE10957_B_model/ (stored 0%)
  adding: 2018EE10957_B_model/checkpoint-1700/ (stored 0%)
  adding: 2018EE10957_B_model/checkpoint-1700/rng_state.pth (deflated 27%)
  adding: 2018EE10957_B_model/checkpoint-1700/training_args.bin (deflated 48%)
  adding: 2018EE10957_B_model/checkpoint-1700/optimizer.pt (deflated 24%)
  adding: 2018EE10957_B_model/checkpoint-1700/config.json (deflated 49%)
  adding: 2018EE10957_B_model/checkpoint-1700/scheduler.pt (deflated 50%)
  adding: 2018EE10957_B_model/checkpoint-1700/pytorch_model.bin (deflated 7%)
  adding: 2018EE10957_B_model/checkpoint-1700/trainer_state.json (deflated 73%)
